In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext lab_black
%matplotlib inline

In [2]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12897764751107957597
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4937233203
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6880477865527730477
physical_device_desc: "device: 0, name: GeForce GTX 1060 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [3]:
import tensorflow as tf

tf.debugging.set_log_device_placement(True)

# Create some tensors
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)

print(c)

Executing op MatMul in device /job:localhost/replica:0/task:0/device:GPU:0
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


# U-NET

In [4]:
import numpy as np
import scipy as scipy
import os
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend as K
from helpers import *

In [5]:
# Import all the necessary for our model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import (
    Activation,
    Dropout,
    Flatten,
    Dense,
    Conv2D,
    MaxPooling2D,
    LeakyReLU,
)
from tensorflow.compat.v2.keras.layers import BatchNormalization
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

In [6]:
from keras_tqdm import TQDMNotebookCallback

# keras, model definition...
cb = TQDMNotebookCallback()
setattr(cb, "on_train_batch_begin", lambda x, y: None)
setattr(cb, "on_train_batch_end", lambda x, y: None)

# model.fit(X_train, Y_train, verbose=0, callbacks=[cb])

Using TensorFlow backend.


# Loading images

In [24]:
# Load a set of images
root_dir = "data/"

# Select the directory for the images and load them
image_dir_train = root_dir + "training/images/"
files = os.listdir(image_dir_train)
n_train = len(files)

print("Loading " + str(n_train) + " images")
imgs_train = np.asarray(
    [load_image(image_dir_train + files[i]) for i in range(n_train)]
)

# Select the directory for groundtruth images and load them
gt_dir_train = root_dir + "training/groundtruth/"
print("Loading " + str(n) + " groundtruth images")
gt_imgs_train = np.asarray(
    [load_image(gt_dir_train + files[i]) for i in range(n_train)]
)

Loading 90 images
Loading 100 groundtruth images


In [25]:
imgs_train.shape

(90, 400, 400, 3)

In [26]:
gt_imgs_train.shape

(90, 400, 400)

In [10]:
image_size = 400

# We separate the images from the groundtruth images
img_patches_train = [img_crop(imgs_train[i], image_size, image_size) for i in range(n_train)]
gt_patches_train = [img_crop(gt_imgs_train[i], image_size, image_size) for i in range(n_train)]

# Linearize the list and labeling them X and Y
X_train = np.asarray(
    [
        img_patches[i][j]
        for i in range(len(img_patches_train))
        for j in range(len(img_patches_train[i]))
    ]
)
Y_train = np.asarray(
    [
        gt_patches[i][j]
        for i in range(len(gt_patches_train))
        for j in range(len(gt_patches_train[i]))
    ]
)

In [11]:
X_train.shape

(100, 400, 400, 3)

In [12]:
Y_train.shape

(100, 400, 400)

# Generating mini-batch and running data augmentation

In [13]:
def create_minibatch(X,Y):

    # Fix the seed
    np.random.seed(1)

    # We define the window size of 72, batch size of 100 (empirically chosen)
    # and patch size should correspond to 16
    w_size = 72
    batch_size = 100
    patch_size = 16
    num_images = 100

    while True:
        # Generate one minibatch
        batch_image = np.empty((batch_size, w_size, w_size, 3))
        batch_label = np.empty((batch_size, 2))

        for i in range(batch_size):

            # Select a random index represnting an image
            random_index = np.random.choice(num_images)

            # Width of original image
            width = 400

            # Sample a random window from the image
            random_sample = np.random.randint(w_size // 2, width - w_size // 2, 2)

            # Create a sub image of size 72x72
            sampled_image = X[random_index][
                random_sample[0] - w_size // 2 : random_sample[0] + w_size // 2,
                random_sample[1] - w_size // 2 : random_sample[1] + w_size // 2,
            ]

            # Take its corresponding ground-truth image
            correspond_ground_truth = Y[random_index][
                random_sample[0] - patch_size // 2 : random_sample[0] + patch_size // 2,
                random_sample[1] - patch_size // 2 : random_sample[1] + patch_size // 2,
            ]

            # We set in the label depending on the threshold of 0.25
            # The label becomes either 0 or 1 by applying to_categorical with parameter 2
            label = to_categorical(
                (np.array([np.mean(correspond_ground_truth)]) > 0.25) * 1, 2
            )

            # The image augmentation is based on both flipping and rotating (randomly in steps of 45°)
            # Random vertical and horizontal flip
            if np.random.choice(2) == 1:
                sampled_image = np.flipud(sampled_image)

            if np.random.choice(2) == 1:
                sampled_image = np.fliplr(sampled_image)

            # Random rotation in steps of 45°
            rotations = [0, 45, 90, 135, 180, 225, 270, 315]

            # We select a rotation degree randomly
            rotation_choice = np.random.choice(len(rotations))

            # Rotate it using the random value (uses the scipy library)
            sampled_image = scipy.ndimage.rotate(
                sampled_image,
                rotations[rotation_choice],
                order=1,
                reshape=False,
                mode="reflect",
            )

            # We put in the sub image and its corresponding label before yielding it
            batch_image[i] = sampled_image
            batch_label[i] = label

        # Yield the mini_batch to the model
        yield (batch_image, batch_label)

# Creating the class (Same as in cnn_model.py, but provided here for better readability)

In [14]:
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

In [15]:
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def recall(y_true, y_pred):
    """Recall metric.

    Only computes a batch-wise average of recall.

    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [22]:
class cnn_model:

    # Initialize the class
    def __init__(self, shape, batch_normalization, activation):
        self.shape = shape
        self.batch_normalization = batch_normalization
        self.activation = activation
        self.model = self.initialize_cnn_model(shape, batch_normalization, activation)

    def initialize_cnn_model(self, shape, batch_normalization, activation):
        #         print(activation)

        # INPUT
        # shape     - Size of the input images
        # OUTPUT
        # model    - Compiled CNN

        # Define hyperparamters
        KERNEL3 = (3, 3)
        KERNEL5 = (5, 5)

        # Define a model
        model = Sequential()

        # Add the layers
        # Selection of the model is described in the report
        # We use padding = 'same' to avoid issues with the matrix sizes
        model.add(Conv2D(64, KERNEL5, input_shape=shape, padding="same"))
        if batch_normalization:
            model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.1)) if activation == "LeakyReLU" else model.add(
            Activation(activation)
        )
        model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
        model.add(Dropout(0.3))

        model.add(Conv2D(128, KERNEL3, padding="same"))
        if batch_normalization:
            model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.1)) if activation == "LeakyReLU" else model.add(
            Activation(activation)
        )
        model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
        model.add(Dropout(0.3))

        model.add(Conv2D(256, KERNEL3, padding="same"))
        if batch_normalization:
            model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.1)) if activation == "LeakyReLU" else model.add(
            Activation(activation)
        )
        model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
        model.add(Dropout(0.3))

        model.add(Conv2D(256, KERNEL3, padding="same"))
        if batch_normalization:
            model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.1)) if activation == "LeakyReLU" else model.add(
            Activation(activation)
        )
        model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
        model.add(Dropout(0.3))

        model.add(Conv2D(256, KERNEL3, padding="same"))
        if batch_normalization:
            model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.1)) if activation == "LeakyReLU" else model.add(
            Activation(activation)
        )
        model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
        model.add(Dropout(0.3))

        model.add(Conv2D(256, KERNEL3, padding="same"))
        if batch_normalization:
            model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.1)) if activation == "LeakyReLU" else model.add(
            Activation(activation)
        )
        model.add(MaxPooling2D(pool_size=(2, 2), padding="same"))
        model.add(Dropout(0.3))

        # Flatten it and use regularizers to avoid overfitting
        # The parameters have been chosen empirically
        model.add(Flatten())
        model.add(
            Dense(
                128, kernel_regularizer=l2(0.000001), activity_regularizer=l2(0.000001)
            )
        )
        if batch_normalization:
            model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.1)) if activation == "LeakyReLU" else model.add(
            Activation(activation)
        )
        model.add(Dropout(0.4))

        # Add output layer
        model.add(
            Dense(2, kernel_regularizer=l2(0.000001), activity_regularizer=l2(0.000001))
        )
        model.add(Activation("sigmoid"))

        # Compile the model using the binary crossentropy loss and the Adam optimizer for it
        # We used the accuracy as a metric, but F1 score is also a plausible choice
        model.compile(
            loss="binary_crossentropy",
            optimizer=Adam(lr=0.001),
            metrics=["accuracy", recall, f1],
        )

        # Print a summary of the model to see what has been generated
        model.summary()

        return model

    def train(self):

        # Early stopping callback after 10 steps
        early_stopping = EarlyStopping(
            monitor="loss", min_delta=0, patience=10, verbose=1, mode="auto"
        )

        # Reduce learning rate on plateau after 4 steps
        lr_callback = ReduceLROnPlateau(
            monitor="loss", factor=0.5, patience=4, verbose=1, mode="auto"
        )

        # Place the callbacks in a list to be used when training
        #         callbacks = [cb, early_stopping, lr_callback]
        callbacks = [early_stopping, lr_callback]

        # Train the model using the previously defined functions and callbacks
        self.model.fit_generator(
            create_minibatch(X_train, Y_train),
            steps_per_epoch=STEPS_PER_EPOCH,
            epochs=EPOCHS,
            use_multiprocessing=False,
            workers=1,
            callbacks=callbacks,
            verbose=1,
            validation_data=create_minibatch(X_val, Y_val),
        )

    def classify(self, X):
        # Subdivide the images into blocks with a stride and patch_size of 16
        img_patches = create_patches(X, 16, 16, padding=28)

        # Predict
        predictions = self.model.predict(img_patches)
        predictions = (predictions[:, 0] < predictions[:, 1]) * 1

        # Regroup patches into images
        return group_patches(predictions, X.shape[0])

    def load(self, filename):
        # Load the model (used for submission)
        dependencies = {
            "recall": recall,
            "f1": f1,
        }
        self.model = load_model(filename, custom_objects=dependencies)

    def save(self, filename):
        # Save the model (used to then load to submit)
        self.model.save(filename)

In [23]:
# Instantiate the model with the size 72x72, the window size of the images to be fed
# We define the number of epochs and steps per epochs
EPOCHS = 200
STEPS_PER_EPOCH = 150
batch_normalization = False
activation = "LeakyReLU"
model = cnn_model(
    shape=(72, 72, 3), batch_normalization=batch_normalization, activation=activation
)
# Train the model
model.train()
model.save("no_batch_LeakyRelu.h5")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 72, 72, 64)        4864      
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 72, 72, 64)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 36, 36, 64)        0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 36, 36, 128)       73856     
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 36, 36, 128)       0         
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 18, 18, 128)      

NameError: name 'valGenerator' is not defined

In [ ]:
# Instantiate the model with the size 72x72, the window size of the images to be fed
batch_normalization = False
activation = "relu"
model = cnn_model(
    shape=(72, 72, 3), batch_normalization=batch_normalization, activation=activation
)
# Train the model with batch
model.train()
model.save("no_batch_relu.h5")

In [ ]:
# Instantiate the model with the size 72x72, the window size of the images to be fed
batch_normalization = True
activation = "LeakyReLU"
model = cnn_model(
    shape=(72, 72, 3), batch_normalization=batch_normalization, activation=activation
)
# Train the model with batch
model.train()
model.save("batch_LeakyReLU.h5")

In [ ]:
# Instantiate the model with the size 72x72, the window size of the images to be fed
batch_normalization = True
activation = "relu"
model = cnn_model(
    shape=(72, 72, 3), batch_normalization=batch_normalization, activation=activation
)
# Train the model with batch
model.train()
model.save("batch_relu.h5")

In [ ]:
from helpers import *

# from cnn_model import cnn_model

# Instantiate the model
batch_normalization = True
activation = "relu"
model = cnn_model(
    shape=(72, 72, 3), batch_normalization=batch_normalization, activation=activation
)

# Load the model
model.load("batch_relu.h5")

# Print a summary to make sure the correct model is used
model.model.summary()

# We add all test images to an array, used later for generating a submission
image_filenames = []
for i in range(1, 51):
    image_filename = "data/test_set_images/test_" + str(i) + "/test_" + str(i) + ".png"
    image_filenames.append(image_filename)

# Set-up submission filename
submission_filename = "batch_relu.csv"

# Generates the submission
generate_submission(model, submission_filename, *image_filenames)